In [0]:
import pandas as pd
import numpy as np

import xgboost as xgb

from sklearn.metrics import mean_absolute_error as mae
from sklearn.model_selection import cross_val_score

from hyperopt import hp, fmin, tpe, STATUS_OK,STATUS_FAIL

import eli5
from eli5.sklearn import PermutationImportance

In [4]:
cd "/content/drive/My Drive/Colab Notebooks/dw_matrix/matrix_two/dw_matrix_car"

/content/drive/My Drive/Colab Notebooks/dw_matrix/matrix_two/dw_matrix_car


In [5]:
df = pd.read_hdf('data/car.h5')
df.shape

(106494, 155)

## Feature engineering

In [0]:
SUFFIX_CAT='__cat'
for feat in df.columns:
  if isinstance(df[feat][0],list): continue

  factorized_values = df[feat].factorize()[0]
  if SUFFIX_CAT in feat:
    df[feat] = factorized_values
  else:
    df[feat+SUFFIX_CAT] = factorized_values

In [0]:
df['param_rok-produkcji']=df['param_rok-produkcji'].map(lambda x: -1 if str(x)=='None' else int(x))
df['param_moc_ext'] = df['param_moc'].map(lambda x: -1 if str(x)=='None' else int(str(x).split(' ')[0]))
df['param_pojemność-skokowa'] = df['param_pojemność-skokowa'].map(lambda x: -1 if str(x)=='None' else int(str(x).split('cm')[0].replace(' ','')))

In [0]:
def run_model(model,feats):
  X = df[feats].values
  y = df['price_value'].values

  scores = cross_val_score(model,X,y,cv=3,scoring='neg_mean_absolute_error')
  return np.mean(scores), np.std(scores)

In [9]:
feats = ['param_napęd__cat','param_rok-produkcji','param_stan__cat','param_skrzynia-biegów__cat','param_faktura-vat__cat','param_moc_ext','param_marka-pojazdu__cat','feature_kamera-cofania__cat','param_typ__cat','param_pojemność-skokowa','seller_name__cat','feature_wspomaganie-kierownicy__cat','param_model-pojazdu__cat','param_wersja__cat','param_kod-silnika__cat','feature_system-start-stop__cat','feature_asystent-pasa-ruchu__cat','feature_czujniki-parkowania-przednie__cat','feature_łopatki-zmiany-biegów__cat','feature_regulowane-zawieszenie__cat']

xgb_params={
    'max_depth':5,
    'n_estimators':50,
    'learning_rate':0.1,
    'seed':0,
    'tree_method': 'gpu_hist'}
    
run_model(xgb.XGBRegressor(**xgb_params),feats)

[09:04:43] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[09:04:44] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[09:04:44] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.


(-9543.62975799748, 65.18270237326064)

## Hyperopt

In [12]:
def obj_func(params):
  print('training with params: ')
  print(params)

  try:
    mean_mae, score_std = run_model(xgb.XGBRegressor(**params),feats)
    return {'loss':np.abs(mean_mae), 'status':STATUS_OK}
  except:
    return {'loss':0, 'status':STATUS_FAIL}

# space
xgb_reg_params = {
    'learning_rate': hp.choice('learning_rate', np.arange(0.05,0.31,0.05)),
    'max_depth': hp.choice('max_depth', np.arange(5,16,1,dtype=int)),
    'subsample': hp.quniform('subsample',0.5,1,0.05),
    'colsample_bytree': hp.quniform('colsample_bytree', 0.5,1,0.05),
    'objective': 'reg:squarederror',
    'n_estimators': 100,
    'seed':0,
    'tree_method': 'gpu_hist',
}

# run
best = fmin(obj_func, xgb_reg_params, algo=tpe.suggest,max_evals=25)
best

training with params: 
{'colsample_bytree': 0.9, 'learning_rate': 0.05, 'max_depth': 14, 'n_estimators': 100, 'objective': 'reg:squarederror', 'seed': 0, 'subsample': 0.9, 'tree_method': 'gpu_hist'}
training with params: 
{'colsample_bytree': 0.75, 'learning_rate': 0.2, 'max_depth': 15, 'n_estimators': 100, 'objective': 'reg:squarederror', 'seed': 0, 'subsample': 0.8500000000000001, 'tree_method': 'gpu_hist'}
training with params: 
{'colsample_bytree': 0.8, 'learning_rate': 0.2, 'max_depth': 7, 'n_estimators': 100, 'objective': 'reg:squarederror', 'seed': 0, 'subsample': 0.6000000000000001, 'tree_method': 'gpu_hist'}
training with params: 
{'colsample_bytree': 0.6000000000000001, 'learning_rate': 0.2, 'max_depth': 10, 'n_estimators': 100, 'objective': 'reg:squarederror', 'seed': 0, 'subsample': 0.65, 'tree_method': 'gpu_hist'}
training with params: 
{'colsample_bytree': 0.5, 'learning_rate': 0.05, 'max_depth': 10, 'n_estimators': 100, 'objective': 'reg:squarederror', 'seed': 0, 'subsam

{'colsample_bytree': 0.9,
 'learning_rate': 1,
 'max_depth': 8,
 'subsample': 0.9500000000000001}